In [3]:
import pandas as pd
from openai import OpenAI
import threading
from queue import Queue
import traceback
import json
import time
import numpy as np
import pickle
import glob

In [4]:
result = pd.read_pickle('./result/total_review.pkl')
result

,电视剧名称,评论id,result,pid,一级,二级,正负向,依据
7228,180天重启计划,16510360,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,观看了前5集的剪辑版，整体体验很好，感觉像看了一场电影，直观感觉是中等偏上。
7228,180天重启计划,16510360,"{'id': 2, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",2.0,艺术精湛,表演质量,正向,吴越老师真的是一如既往的优秀，虽说不在青春，但她演出来的感觉颇具少女韵味，美丽又可爱。
7228,180天重启计划,16510360,"{'id': 3, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",3.0,艺术精湛,表演质量,正向,陈明昊老师和耿乐老师也是老演员啦，他们的表演收放自如，游刃有余。
7228,180天重启计划,16510360,"{'id': 4, '一级': '艺术精湛', '二级': '表演质量', '正负向': '...",4.0,艺术精湛,表演质量,正向,周雨彤和刘畅两位青年演员在剧里的表现也可圈可点。
7228,180天重启计划,16510360,"{'id': 5, '一级': '艺术精湛', '二级': '人物塑造', '正负向': '...",5.0,艺术精湛,人物塑造,正向,周雨彤在现场表达自己对角色的看法时理解透彻。
...,...,...,...,...,...,...,...,...
779,﻿无尽的尽头,325389301,"{'id': 1, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",1.0,艺术精湛,剧情结构,正向,我二刷发现，霹雳舞其实在他们跟着社工去酒吧那集就被cue到了！
766,﻿无尽的尽头,325972942,"{'id': 1, '一级': '整体评价', '二级': '无具体指向', '正负向': ...",1.0,整体评价,无具体指向,正向,24集。
771,﻿无尽的尽头,326027897,"{'id': 1, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",1.0,艺术精湛,剧情结构,负向,这段我也觉得胡扯。
780,﻿无尽的尽头,327790809,"{'id': 1, '一级': '思想精深', '二级': '社会反思', '正负向': '...",1.0,思想精深,社会反思,正向,因为她那时候知道福利院可以养她女儿了。另外她很小就被万学民控制，对万的害怕不是我们局外人能想象的


In [2]:
long_raw = pd.read_excel("./采集/长评.xlsx")
long_raw

,电视剧名称,电视剧id,影评id,影评人名称,影评时间,评分,标题,有用数,无用数,影评内容
0,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...
1,﻿无尽的尽头,36656706,16688684,尘飞扬,2025-04-28 20:53:32,40.0,《无尽的尽头》：不是为罪行开脱，而是看清罪恶的土壤,139.0,5.0,一口气刷完8集《无尽的尽头》，我会继续追下去。 第五集，白恩宇对检察官郑雁来说的话，可谓掷地...
2,﻿无尽的尽头,36656706,16682513,比万大魔王,2025-04-25 19:18:04,30.0,可惜，痛点太明显,123.0,13.0,《无尽的尽头》聚焦的是未成年人犯罪。 出发点是好的，但就完成度来说，差了些。至少目前首更这几...
3,﻿无尽的尽头,36656706,16692186,薇羅尼卡,2025-04-30 18:16:43,40.0,在破碎中，看见坚持的人性光,53.0,6.0,说实话，点开《无尽的尽头》之前，我并没有抱特别大的期待。未成年人犯罪、检察官题材，这样的标签...
4,﻿无尽的尽头,36656706,16685028,唐山,2025-04-26 23:10:34,40.0,通过《无尽的尽头》，我们看到的是自己,80.0,8.0,如果曾被去年3·10邯郸三名未满14岁的初中生杀害同学案震惊；如果你知道，中国未成年犯罪率正...
...,...,...,...,...,...,...,...,...,...,...
16230,无所畏惧之永不放弃,36660843,16473510,娱娱鱼,2025-02-03 00:19:51,50.0,《无所畏惧2》观后感，许卓对梅大梁夫妇的恨，来自破产清算案,1.0,NaN,《无所畏惧2之永不放弃》观后感，我上篇怀疑许卓跟梅大梁夫妇有私人恩怨，结果是公家恩怨。 所有...
16231,无所畏惧之永不放弃,36660843,16771920,Pepper &amp; CC,2025-06-12 10:51:02,40.0,从心理学看第二季结局——李沅案是狗尾续貂，还是符合人设的大戏,NaN,NaN,假如没有李沅案，陈硕从玩世不恭到非罗英子不可，就是一个简单的爱情泡沫剧。李沅案整合了陈硕和罗...
16232,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...
16233,无所畏惧之永不放弃,36660843,16502090,娱娱鱼,2025-02-13 22:31:13,50.0,《无所畏惧2》结局，坏律师官司缠身，好律师财源广进,NaN,NaN,《无所畏惧2》结局，坏律师官司缠身，好律师财源广进。方丽虹和许卓因魏丞案而受到了反噬。 在罗...


In [6]:
len(set(long_raw['影评id'].unique()) - set(result['评论id'].unique()))

377

In [8]:
long_merge = long_raw.merge(result, left_on='影评id', right_on='评论id', suffixes=('', '_ana'))
long_merge

,电视剧名称,电视剧id,影评id,影评人名称,影评时间,评分,标题,有用数,无用数,影评内容,电视剧名称_ana,评论id,result,pid,一级,二级,正负向,依据
0,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...,﻿无尽的尽头,16703529,"{'id': 1, '一级': '思想精深', '二级': '价值导向', '正负向': '...",1.0,思想精深,价值导向,正向,一群那么优秀的女性，在过去公检法题材里有吗？
1,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...,﻿无尽的尽头,16703529,"{'id': 2, '一级': '思想精深', '二级': '社会反思', '正负向': '...",2.0,思想精深,社会反思,正向,你看就没看就断定无尽会污名化女性？
2,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...,﻿无尽的尽头,16703529,"{'id': 3, '一级': '思想精深', '二级': '主题立意', '正负向': '...",3.0,思想精深,主题立意,正向,我第一次在看到如此严谨的庭审，我第一次明白了原来检察官弄证据这么复杂，我一个超级抵触说教上价...
3,﻿无尽的尽头,36656706,16703529,过季季风,2025-05-06 12:51:20,50.0,我是来骂人的！！,717.0,17.0,对骂的就是你们，为了提高豆瓣账号权重，播出一集就TM给一星，还骂的这么难听。我为什么骂你们，...,﻿无尽的尽头,16703529,"{'id': 4, '一级': '思想精深', '二级': '价值导向', '正负向': '...",4.0,思想精深,价值导向,正向,我一个超级抵触说教上价值的人，我懂了强制报告制度、剥夺监护权、合适成年人、附条件不起诉、刑责年龄。
4,﻿无尽的尽头,36656706,16688684,尘飞扬,2025-04-28 20:53:32,40.0,《无尽的尽头》：不是为罪行开脱，而是看清罪恶的土壤,139.0,5.0,一口气刷完8集《无尽的尽头》，我会继续追下去。 第五集，白恩宇对检察官郑雁来说的话，可谓掷地...,﻿无尽的尽头,16688684,"{'id': 1, '一级': '思想精深', '二级': '主题立意', '正负向': '...",1.0,思想精深,主题立意,正向,作为一部聚焦未成年人司法保护的悬疑剧，《无尽的尽头》不仅拍未成年人的犯罪事件，而且在拍未成年...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102144,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...,无所畏惧之永不放弃,16500663,"{'id': 6, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",6.0,艺术精湛,剧情结构,负向,好好 的梅大梁案从第一部渲染到第二部，就这么一集草草结束，说白了第一季就女主被前夫坑再引出梅...
102145,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...,无所畏惧之永不放弃,16500663,"{'id': 7, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",7.0,艺术精湛,剧情结构,负向,许建设坠车逃跑他把狱警（押犯警察）拖离坠车那我以为是个伏笔，以为主角在后面给他辩护时会用到呢...
102146,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...,无所畏惧之永不放弃,16500663,"{'id': 8, '一级': '艺术精湛', '二级': '剧情结构', '正负向': '...",8.0,艺术精湛,剧情结构,负向,你两女主不是打破沙锅坚持正义吗？霸凌案槽点过多，施暴者脸谱化，男主手机录像取证无语，给人感觉...
102147,无所畏惧之永不放弃,36660843,16500663,Joe Chong G,2025-02-13 03:59:49,10.0,主角律师们,NaN,NaN,都好厉害好有手段呢，夏舒在这外国有个旧同学，在那外国也有个旧同学能查到资料，其他律所的律师们...,无所畏惧之永不放弃,16500663,"{'id': 9, '一级': '思想精深', '二级': '社会反思', '正负向': '...",9.0,思想精深,社会反思,负向,现实霸凌比这残酷多了别说什么电视受众审核什么的，编得就是差，设定和细节也支撑不起来配合那演技...


In [9]:
short_raw = pd.read_excel('./采集/短评.xlsx')
short_raw

,电视剧名称,电视剧id,短评id,评论人,评论时间,评论点赞数,位置,评论内容,评分
0,﻿无尽的尽头,36656706,4520292443,反正都会后悔,2025-05-01 01:24:15,4449,广东,发现有些国人真挺好笑的，比如韩国一些现实题材通过电影影视呈现出来，就会说好厉害好敢拍，不像我...,40.0
1,﻿无尽的尽头,36656706,4515142798,小熊饼干脆脆面,2025-04-24 19:53:48,3107,陕西,作为一个法学专业毕业的，实话说把庭审流程拍对的不多，这个算一个,50.0
2,﻿无尽的尽头,36656706,4517572691,冰冰冰冰冰淇淋,2025-04-27 17:33:04,3344,四川,看了8集之后我觉得第一天就冲进来给一星的可能多少都有点反社会,40.0
3,﻿无尽的尽头,36656706,4515176107,燚见电影,2025-04-24 20:38:02,2206,河南,黄家旺这么标准的incel不可能悔过的，建议关到死。,40.0
4,﻿无尽的尽头,36656706,4516055933,Moss.Y,2025-04-25 22:33:32,2755,山东,能把现实拆解开拍好了以警醒世人就不错了，咋？给你们拍一个平行世界，那个世界里的法律和现在的不...,50.0
...,...,...,...,...,...,...,...,...,...
7964,无所畏惧之永不放弃,36660843,4437840298,蚊蚊,2025-02-05 16:21:53,5,广东,编剧你不想有感情线就不要写嘛，搞得这部好像就是为了这份感情所续的，最后还是想让人家分开。,20.0
7965,无所畏惧之永不放弃,36660843,4466921507,麦伦,2025-03-02 19:15:46,3,广东,这居然还是第二部……震惊……,20.0
7966,无所畏惧之永不放弃,36660843,4451640881,equus,2025-02-17 01:36:12,10,北京,传统剧两大法宝，让角色蠢死，让观众气死,10.0
7967,无所畏惧之永不放弃,36660843,4450190399,乔,2025-02-15 23:18:05,4,北京,忘了第一部了，慢慢回忆起来的。总的说应该是没大毛病。但剧情集中在事务所内部的悬疑失去了律师剧...,20.0


In [11]:
len(set(short_raw['短评id'].unique()) - set(result['评论id'].unique()))

1838

In [12]:
temp = pd.read_csv('./clean/short.csv')
temp

,评论id,评论内容
0,4452800852,周雨彤演技就是合格 一用力就尴尬
1,4452553100,第一集离婚剧情没有前因后果安排的也太不合理了吧
2,4452810042,周雨彤一人分饰两角挑战很大但是演绎效果很好，看过原著小说，希望后续继续好好刻画母女关系
3,4452868355,李漠这次拍的好失败。每10分钟就回忆部分和现在的剧情穿插，搞得哪部分都看不进去。母亲年轻时候...
4,4452678015,没有必要设置老妈怀孕这种情节
...,...,...
6386,4518592097,男高偷拍传播yhsq作为检察官说这很正常还一直在洗666
6387,4517222924,没有这种深度尺度就不要学人少年法庭拍这种剧了，打着聚焦未成年犯罪的噱头…
6388,4519184797,唯一的亮点就是未检题材。
6389,4533842320,电视剧想表现什么？受害者被遗忘，加害者在监狱开心的玩泥巴，受害者的母亲变泼妇。太可怕了。没有...
